https://trello.com/c/2ukxM5oY/2-alabama-icu-metric-is-broken-in-303

In [ ]:
%load_ext autoreload
%autoreload 2
from libs.datasets import DHBeds
from libs.datasets import CovidCareMapBeds
from libs.datasets import JHUDataset
from libs.datasets.sources.nha_hospitalization import NevadaHospitalAssociationData
from libs.datasets import NYTimesDataset
from libs.datasets import CovidTrackingDataSource
from libs.datasets import CDSDataset
from libs.datasets import FIPSPopulation
from libs.datasets.dataset_utils import AggregationLevel
from libs.enums import Intervention
from libs.datasets import combined_datasets
# from libs.datasets.sources.interventions import StateInterventionDataSource

from libs.datasets import dataset_utils
from libs.build_processed_dataset import get_testing_timeseries_by_state
from libs.datasets import custom_aggregations
import pathlib
import json
import numpy as np
import pandas as pd
import seaborn as sns
import requests
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

In [ ]:
STATE = "LA"
SNAPSHOTS = [408, 415]

In [ ]:
covid_tracking_ts = CovidTrackingDataSource.local().timeseries()
covid_tracking_ts.get_subset(AggregationLevel.STATE, state=STATE, after="2020-05-29").data

# Check the Links on Compare

In [ ]:
print(f'https://covidactnow.org/compare?left={SNAPSHOTS[0]}&metric=2&right={SNAPSHOTS[1]}&sort=2')

# Check the API Data

In [ ]:
for snapshot in SNAPSHOTS:
    version = requests.get(f'https://data.covidactnow.org/snapshot/{snapshot}/version.json')
    summary = requests.get(f'https://data.covidactnow.org/snapshot/{snapshot}/us/states/{STATE}.OBSERVED_INTERVENTION.json')
    print(snapshot)
    print(version.json())
    print(summary.json())

In [ ]:
summary.json()

In [ ]:
for key, item in summary.json()['actuals']['ICUBeds'].items():
    print(key, type(item))

In [ ]:
def get_snapshot_summaries(snapshots):
    url = 'https://data.covidactnow.org/snapshot/{snapshot}/us/states/{state}.OBSERVED_INTERVENTION.json'
    data = [requests.get(url.format(snapshot=snapshot, state=STATE)).json() for snapshot in snapshots]
    return data

In [ ]:
def compare_snapshot_summaries(snapshots, verbose=False):
    """Traverse first and print comparison to following"""
    data = get_snapshot_summaries(snapshots)
    a = data[0]
    b = data[-1]
    walk_two(a, b, verbose=verbose)        

In [ ]:
def walk(node):
    for key, item in node.items():
        if isinstance(item, dict):
            walk(item)
        else:
            print(key,item)
            
def walk_two(node1, node2, verbose=False):
    for key in node1.keys():
        if isinstance(node1[key], dict):
            print(key)
            walk_two(node1[key], node2[key], verbose=verbose)
        else:
            if verbose:
                print(key, node1[key], node2[key])
            else:
                if node1[key] != node2[key]:
                    print(key, node1[key], node2[key])

In [ ]:
compare_snapshot_summaries(SNAPSHOTS, verbose=False)

In [ ]:
def get_df_from_snapshot(state, snapshot):
    """"""
    r = requests.get(f'https://data.covidactnow.org/snapshot/{snapshot}/us/states/{STATE}.OBSERVED_INTERVENTION.timeseries.json')
    df = pd.DataFrame(r.json()['timeseries'])
    df['date'] = pd.to_datetime(df['date'])
    df['snapshot'] = snapshot
    return df

In [ ]:
dfs = pd.concat([get_df_from_snapshot(STATE, s) for s in SNAPSHOTS])

In [ ]:
dfs.columns

In [ ]:
fig, ax = plt.subplots()
for num, data in dfs.groupby('snapshot'):
    data.plot(x='date',y='ICUBedsInUse',ax=ax, label=num, marker='x')
ax.axvline(pd.to_datetime('today'), linestyle='--', color='black', label='Today')
ax.legend()
ax.set(ylabel='ICUBedsInUse', title=f'Extrapolated ICU Estimates for {STATE}')
fig.autofmt_xdate()

In [ ]:
dfs['snapshot'].unique()

In [ ]:
# for snapshot in SNAPSHOTS:
#     version = requests.get(f'https://data.covidactnow.org/snapshot/{snapshot}/version.json')
#     summary = requests.get(f'https://data.covidactnow.org/snapshot/{snapshot}/us/states/{STATE}.OBSERVED_INTERVENTION.timeseries.json')
#     print(snapshot)
#     print(version.json())
#     print(summary.json())

In [ ]:
# pd.DataFrame(summary.json()['timeseries'])

# Scrap Paper